In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc

# Parameters
num_symbols = 100000  # Number of symbols (increase for better accuracy at low BER)
snr_db = np.arange(0, 21, 2)  # SNR range in dB
ber = np.zeros(len(snr_db))  # To store BER values

# BPSK Modulation
def bpsk_modulation(bits):
    return 2 * bits - 1  # Map 0 -> -1, 1 -> 1

# BPSK Demodulation
def bpsk_demodulation(received_signal):
    return received_signal.real > 0  # Decision rule

# AWGN Channel
def awgn_channel(signal, snr_db):
    snr_linear = 10 ** (snr_db / 10)
    noise_power = 1 / snr_linear
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape))
    return signal + noise

# Simulation
for i, snr in enumerate(snr_db):
    # Generate random data (0s and 1s)
    data = np.random.randint(0, 2, num_symbols)
    
    # BPSK Modulation
    modulated_signal = bpsk_modulation(data)
    
    # Pass through AWGN Channel
    received_signal = awgn_channel(modulated_signal, snr)
    
    # BPSK Demodulation
    demodulated_data = bpsk_demodulation(received_signal)
    
    # Calculate BER
    errors = np.sum(data != demodulated_data)
    ber[i] = errors / num_symbols
    
    # Stop simulation if BER is below 1e-6 (to save time)
    if ber[i] < 1e-6:
        break

# Theoretical BER for BPSK (for comparison)
theoretical_ber = 0.5 * erfc(np.sqrt(10 ** (snr_db / 10)))

# Plotting
plt.figure(figsize=(10, 6))
plt.semilogy(snr_db[:len(ber)], ber, 'bo-', label='Simulated BER (BPSK)')
plt.semilogy(snr_db, theoretical_ber, 'r--', label='Theoretical BER (BPSK)')
plt.xlabel('SNR (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('BER vs SNR for BPSK Modulation')
plt.grid(True, which="both", ls="--")
plt.ylim(1e-6, 1)  # Set BER limits
plt.legend()
plt.show()